In [1]:
import sys, os
parent_dir = os.getcwd()
path = os.path.dirname(parent_dir)
sys.path.append(path)

from gym_homer.envs.homer_env import HomerEnv
import pandas as pd
import numpy as np

import torch
import gym
from gym import spaces, wrappers
import envpool

from tianshou.utils import WandbLogger
from torch.utils.tensorboard import SummaryWriter

from tianshou.data import Collector, VectorReplayBuffer, AsyncCollector
from tianshou.env import DummyVectorEnv, SubprocVectorEnv, ShmemVectorEnv
from tianshou.policy import PPOPolicy
from tianshou.trainer import onpolicy_trainer
from tianshou.utils.net.common import ActorCritic, Net
from tianshou.utils.net.discrete import Actor, Critic

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
i=0

In [2]:
path

'/home/ds1/w210/home-energy-optimizer'

In [3]:
log_path = path+'/logs/'
logger = WandbLogger(
    project="RL_project", 
    entity="w266_wra",
    name=f'debug_{i}',
    config=None)
logger.load(SummaryWriter(log_path))
i += 1

wandb: Currently logged in as: atox120 (w266_wra). Use `wandb login --relogin` to force relogin


wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /home/ds1/w210/home-energy-optimizer/logs/


In [15]:
# Load Data
data = pd.read_csv(path+"/test_env_data.csv", index_col=False).fillna(0)
vectorised = False


n_train_envs = 10
n_test_envs = 10

env = HomerEnv(data=data, start_soc='full')
train_envs = SubprocVectorEnv([lambda: HomerEnv(
    data=data, 
    capacity=10,
    start_soc='full', 
    discrete=True,
    charge_rate = 12) for _ in range(n_train_envs)])

test_envs = SubprocVectorEnv([lambda: HomerEnv(
    data=data, 
    capacity=10,
    start_soc='full', 
    discrete=True,
    charge_rate = 12,
    save_history=True,
    save_path=path) for _ in range(n_test_envs)])

In [16]:
data

,time_x,time_y,weekend,month_x,month_y,region_1,region_2,region_3,solar,loads,import_tariff,export_tariff,max_d,max_c,soc
0,0.000,1.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
1,0.500,0.866,0,0.0,1.0,0,0,0,-2,0.5,0.05,0.0,0.0,0.0,0.0
2,0.866,0.500,0,0.0,1.0,0,0,0,-2,0.5,0.05,0.0,0.0,0.0,0.0
3,1.000,0.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
4,0.866,-0.500,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
5,0.500,-0.866,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
6,0.000,-1.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
7,-0.500,-0.866,0,0.0,1.0,0,0,0,0,0.5,2.00,2.0,0.0,0.0,0.0
8,-0.866,-0.500,0,0.0,1.0,0,0,0,0,0.5,2.00,2.0,0.0,0.0,0.0
9,-1.000,-0.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0


In [17]:
env.action_space.n

3

In [18]:
# net is the shared head of the actor and the critic
hidden_sizes = [64,64]
lr_optimizer = 1e-4

net = Net(
    env.observation_space.shape, 
    hidden_sizes=hidden_sizes, 
    device=device
    )

actor = Actor(net, 3, device=device).to(device)
#actor = Actor(net, env.action_space.n, device=device).to(device)

critic = Critic(net, device=device).to(device)
actor_critic = ActorCritic(actor, critic)

# optimizer of the actor and the critic
optim = torch.optim.Adam(actor_critic.parameters(), lr=lr_optimizer)

In [19]:
print(env.action_space)

Discrete(3)


In [20]:
# Since environment action space is discrete 
dist = torch.distributions.Categorical
policy = PPOPolicy(
    actor, 
    critic, 
    optim, 
    dist, 
    action_space=env.action_space, 
    deterministic_eval=True
)

In [21]:
vectorised = False

if vectorised:
    train_collector = AsyncCollector(policy, train_envs, VectorReplayBuffer(20000, len(train_envs)))
    test_collector = AsyncCollector(policy, test_envs)
else:
    train_collector = Collector(policy, train_envs, VectorReplayBuffer(20000, len(train_envs)))
    test_collector = Collector(policy, test_envs)

In [22]:
n_steps = 5000
n_max_epochs = 20
rep_per_collector = 10
eps_per_test = 10
batch_size = 256
n_steps_per_collect = 2000
reward_stop = 4

result = onpolicy_trainer(
    policy,
    train_collector,
    test_collector,
    max_epoch=n_max_epochs,
    step_per_epoch=n_steps,
    repeat_per_collect=rep_per_collector,
    episode_per_test=eps_per_test,
    batch_size=batch_size,
    step_per_collect= n_steps_per_collect,
    stop_fn=lambda mean_reward: mean_reward >= reward_stop,
    logger=logger,
    verbose=True
)

Epoch #1: 6000it [00:00, 8122.64it/s, env_step=6000, len=11, loss=1.274, loss/clip=-0.008, loss/ent=1.089, loss/vf=2.587, n/ep=180, n/st=2000, rew=-1.71]                                                       


Epoch #1: test_reward: 1.974342 ± 0.000000, best_reward: 1.974342 ± 0.000000 in #1


Epoch #2: 6000it [00:00, 8267.17it/s, env_step=12000, len=11, loss=1.165, loss/clip=-0.017, loss/ent=0.936, loss/vf=2.384, n/ep=190, n/st=2000, rew=-0.30]                                                      


Epoch #2: test_reward: 1.974342 ± 0.000000, best_reward: 1.974342 ± 0.000000 in #1


Epoch #3: 6000it [00:00, 8180.79it/s, env_step=18000, len=11, loss=0.637, loss/clip=-0.013, loss/ent=0.635, loss/vf=1.314, n/ep=180, n/st=2000, rew=1.07]                                                       


...

Epoch #18: test_reward: 2.000000 ± 0.000000, best_reward: 2.000000 ± 0.000000 in #15


Epoch #19: 6000it [00:00, 8318.83it/s, env_step=114000, len=11, loss=-0.003, loss/clip=-0.002, loss/ent=0.200, loss/vf=0.001, n/ep=180, n/st=2000, rew=1.99]                                                    


Epoch #19: test_reward: 2.000000 ± 0.000000, best_reward: 2.000000 ± 0.000000 in #15


Epoch #20: 6000it [00:00, 8277.41it/s, env_step=120000, len=11, loss=-0.003, loss/clip=-0.001, loss/ent=0.171, loss/vf=0.001, n/ep=180, n/st=2000, rew=1.99]                                                    

Epoch #20: test_reward: 2.000000 ± 0.000000, best_reward: 2.000000 ± 0.000000 in #15


In [25]:
# Let's watch its performance!
policy.eval()
result = test_collector.collect(n_episode=1, render=False)
print("Final reward: {}, length: {}".format(result["rews"].mean(), result["lens"].mean()))

Final reward: 2.0, length: 11.0


In [26]:
print(result)

{'n/ep': 1, 'n/st': 11, 'rews': array([2.]), 'lens': array([11]), 'idxs': array([0]), 'rew': 2.0, 'len': 11.0, 'rew_std': 0.0, 'len_std': 0.0}


In [27]:
## Not in dcs - also not sure if required. 
logger.wandb_run.finish()